In [7]:
import torch
import matplotlib.pyplot as plt
import BVP_function as bvp
import numpy as np
import deepxde as dde

import seaborn as sns
sns.set_style("whitegrid") # better looking plots

print("Using GPU:", torch.cuda.is_available())
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
if torch.cuda.is_available():
    torch.set_default_tensor_type(torch.cuda.FloatTensor)

from importlib import reload
reload(bvp)

Using GPU: False


<module 'BVP_function' from 'c:\\Users\\u0150568\\OneDrive - KU Leuven\\Code\\public github\\pinn-cm\\inverse\\BVP_function.py'>

In [8]:
lmbd = 1.0
lmbd_trainable = dde.Variable(lmbd - 0.2)
mu = 0.5
mu_trainable = dde.Variable(mu + 0.2)
Q = 4.0

domain = np.array([[0.0, 1.0], [0.0, 1.0]])
geom = dde.geometry.Rectangle([0, 0], [1, 1])

phy_params = {'lmbd': lmbd, 'mu': mu, 'Q': Q}
phy_params_trainable = {'lmbd': lmbd_trainable, 'mu': mu_trainable, 'Q': Q}

#net parameters
net_type = ['Unet','USnet'][1]
n_layers = 4
size_layers = 50
activation = ['tanh','ReLU','Sigmoid'][0]

#loss parameters
loss_type = ['pde','energy'][0] #the energy loss appears to crash LBFGS and provide less accurate results with Adam. PDE loss is therefore used in the following
num_domain = 50**2
train_distribution = ['uniform','pseudo','LHS','Halton','Hammersley','Sobol'][4]

#boundary conditions
bc_type = ['soft','hard'][1]
num_boundary = 50

#optimizer
optimizers = ["adam","L-BFGS"]
learning_rates = [1e-3,None]
iterations = [3000,None]

config = {'net_type':net_type,'n_layers':n_layers,'size_layers':size_layers,'activation':activation,
        'loss_type':loss_type,'num_domain':num_domain,'train_distribution':train_distribution,
        'bc_type':bc_type,'num_boundary':num_boundary,
        'optimizers':optimizers,'learning_rates':learning_rates,'iterations':iterations}  

net_exact = bvp.set_exact_solution(net_type, phy_params)

model, net_wrong_elasticity, pde_net, energy_net, mat_net = bvp.model_setup(geom, config, phy_params_trainable)


RuntimeError: Can't call numpy() on Tensor that requires grad. Use tensor.detach().numpy() instead.

In [ ]:
trainable_variables = [lmbd_trainable, mu_trainable]
variable = dde.callbacks.VariableValue(trainable_variables, period=100, filename="elasticity_param.dat")

model.compile("adam", lr=0.001, external_trainable_variables=trainable_variables, metrics=["l2 relative error"])
losshistory, train_state = model.train(iterations=3000,display_every=200, callbacks=[variable])

# # train lbfgs
model.compile("L-BFGS", external_trainable_variables=trainable_variables, metrics=["l2 relative error"])
losshistory, train_state = model.train(callbacks=[variable])
print(f"lambda:{lmbd_trainable} ; mu: {lmbd_trainable}")